# Import Packages

In [1]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from colorama import Fore,Back,Style
from keras.callbacks import EarlyStopping

# Load Dataset

In [2]:
with open("intents.json") as f:   #load json file containing intents
    data = json.load(f)

# Preprocess Data

In [3]:
# Initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

words = []
classes = []
doc_X = []
doc_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)   # tokenize each patterns
        words.extend(tokens)                   # add tokens of words to list 'words'
        doc_X.append(pattern)                  # append patterns
        doc_y.append(intent["tag"])            # append tags 
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words and convert them to lowercase if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))            # remove duplicate words and sort them in alphabetical order
classes = sorted(set(classes))        # remove duplicate tags and sort them in alphabetical order

print("Length of doc_X :",len(doc_X))
print("Length of doc_X :",len(doc_y))
print("Number of unique words in patterns :",len(words))
print("Number of tag :",len(classes))

Length of doc_X : 115
Length of doc_X : 115
Number of unique words in patterns : 134
Number of tag : 30


In [4]:
print("Types of Tag",classes)

Types of Tag ['about', 'age', 'ai', 'chatbot', 'clever', 'code', 'complaint', 'computer', 'createaccount', 'cricket', 'earning', 'economics', 'emotion', 'feelings', 'food', 'foodyouprefer', 'goodbye', 'greeting', 'help', 'jokes', 'literature', 'movies', 'name', 'os', 'podbaydoor', 'podbaydoorresponse', 'selfaware', 'stock', 'thanks', 'understandquery']


## Create training data and labels

In [5]:
training = []                       # list for training data
out_empty = [0] * len(classes)      # list for labels
# creating the bag of words
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())           # lematize and lowecase data
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row = list(out_empty)     
    output_row[classes.index(doc_y[idx])] = 1   # mark the index of class that the current pattern is associated to 
    training.append([bow, output_row])        

random.shuffle(training)                     # shuffle the data
training = np.array(training, dtype=object)  # convert  to an array
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Build Model

In [6]:
input_shape = (len(train_X[0]),)       #shape of input data
output_shape = len(train_y[0])         #shape of output data
epochs = 200                        

model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               17280     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1950      
Total params: 27,486
Trainable params: 27,486
Non-trainable params: 0
_________________________________________________________________


In [7]:
callback = EarlyStopping(monitor="loss", patience=7, restore_best_weights=True) 
model.fit(x = train_X, y = train_y, epochs = 200, verbose = 1,callbacks = [callback])

Epoch 1/200
4/4 [==============================] - 1s 3ms/step - loss: 3.4249 - accuracy: 0.0261
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 3.1492 - accuracy: 0.1565
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 2.9113 - accuracy: 0.2000
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 2.5232 - accuracy: 0.3130
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 2.2510 - accuracy: 0.4000
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 1.9231 - accuracy: 0.4870
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 1.5134 - accuracy: 0.6000
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 1.3193 - accuracy: 0.6783
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 1.1488 - accuracy: 0.7304
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9914 - accuracy: 0.7652
Epoch 11/200
4/4 [===========

# Cleaning text data

In [8]:
def clean_text(text):
    """
    Input:
    text = input text
    ___________________________________
    Output:
    tokens = token of words
    ____________________________________
    Objective : to clean the input text
    ____________________________________
    """
    tokens = nltk.word_tokenize(text)                            # tokenize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]     # lemmatize
    return tokens


def bag_of_words(text, vocab):
    """
    Input :
    text  =  input text
    vocab =  all unique words in the input data (patterns)
    ________________________________________________
    Output : returns bag of words
    _________________________________________________
    Objective : convert text into numeric value by checking the occurance of word in the text
    _________________________________________________
    """
    tokens = clean_text(text)        # clean_text function
    bow = [0] * len(vocab)           # empty bag of words with length = number of classes
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w:                
                bow[idx] = 1
    return np.array(bow)


def pred_class(text, vocab, labels):
    """
    Input :
    text   = input text
    vocab  = unique words in input data (patterns)
    labels = different  classes of intents
    ________________________________________________
    Output : returns list of predicted intent classes
    _________________________________________________
    Objective : predicts the intent class using neural network 
    __________________________________________________
    """
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.2                                      # minimum threshold for probability 
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]    #prediction greater than minimum threshold  
    y_pred.sort(key=lambda x: x[1], reverse=True)     # sort probability in reverse order
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])              # takes first index of y_pred which has high probability
    return return_list


def get_response(intents_list, intents_json):
    """
    Input :
    intents_list = Predicted tag name
    intents_json = Input data
    ______________________________________________
    Output :
    result = randomly choosen response
    _______________________________________________
    Objective : get reponse for the predicted tag
    """
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

# Running the Chatbot

In [9]:
print(Fore.RED + "Start messaging with the bot (type 'quit' to stop)!" + Style.RESET_ALL)
while True:
    print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    message = input()
    if (message == "quit"):
        break
    
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(Fore.GREEN + "Bot:" + Style.RESET_ALL ,result)

Start messaging with the bot (type 'quit' to stop)!
User: hi
Bot: Hi there
User: who are you
Bot: I'm Joana, an Artificial Intelligent bot
User: what is a chatbot
Bot: I am a chat bot. I am the original chat bot. Did you know that I am incapable of error?
User: suggest me a scifi movie
Bot: 2001-a space odyssey is a good one 
User: how old are you
Bot: I am 27 years old!
User: what is ai
Bot: Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.
User: tell me a joke
Bot: A three-legged dog walks into a saloon in the Old West. He slides up to the bar and announces: 'I'm looking for the man who shot my paw.'  
User: tell me another joke
Bot: Four fonts walk into a bar the barman says 'Oi - get out! We don't want your type in here'  
User: another joke
Bot: Two cannibals are eating a clown. One says to  the other: 'Does this taste funny to you?'
User: quit
